In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def clean_string(x): 
    return x.contents[0].split('\n')[1].strip()

def get_bourso_info(ISIN):

    url = "https://www.boursorama.com/bourse/trackers/cours/" + ISIN
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    caracs = soup.find("div", {"class": "c-etf-characteristics"})
    h = caracs.findAll("p", {"class": "c-list-info__heading"})
    c = caracs.findAll("p", {"class": "c-list-info__value"})
    s = []
    for (i,j) in zip(h,c):
        try:
            s.append([clean_string(i), clean_string(j)])
        except:
            continue
    s = pd.DataFrame(s, columns = ['heading', ISIN])
    s.set_index('heading', inplace = True)
    
    s.loc['Name'] = soup.find("a", {"class": "c-faceplate__company-link"}).attrs['title']
    s.loc['ISIN'] = soup.find("h2", {"class": "c-faceplate__isin"}).contents[0].split(' ')[0]
    s.loc['Ref'] = soup.find("p", {"class": "c-list-info__value"}).contents[0].split('\n')[1].strip()
    
    s.name = ISIN
    return s

get_bourso_info("FR0013412269")

,FR0013412269
heading,
Date de création,25.04.2019
Promoteur,Amundi Asset Management
Gérants,ND
Catégorie morningstar,Actions Etats-Unis Gdes Cap. Croissance
Forme juridique,FCP
Classe d'actifs,Actions Amérique du Nord
Zone géographique,Etats-Unis
Dividende,ND
Frais d'entrée,ND


In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]


s = []
for i in range(1,11):
    url = "https://www.boursorama.com/bourse/trackers/palmares/page-" + str(i) + "?etfSearch%5Bclass%5D=&etfSearch%5Bpea%5D=1&etfSearch%5BmorningstarCategory%5D=&etfSearch%5Bunderlying%5D=&etfSearch%5BsubmitButton%5D="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    caracs = soup.find("div", {"class": "u-relative"})
    h = caracs.findAll("a", {"class": "c-link--animated"})
    s.append([i.attrs['href'].split('/')[-2] for i in h])
    
etf_list = flatten(s)

In [4]:
s = []
for link in etf_list:
    try:
        s.append(get_bourso_info(link))
    except:
        print(link)
        continue

etf_infos = pd.concat(s, axis = 1).T

In [25]:
# Association ISIN / Symbol grâce aux données EURONEXT

data_euronext = pd.read_csv('data/Euronext_ETPs_2020-04-15.xls', sep = "\t")
data_euronext.set_index('ISIN', inplace= True)
etf_infos['ISIN'] = etf_infos['ISIN'].apply(str)
etf_infos.set_index('ISIN', inplace = True)

etf_total = etf_infos.join(data_euronext, how = 'left', rsuffix='euronext')
etf_total['Symbol'] = etf_total['Symbol'] + '.PA'
etf_total.info()

,Name,Symbol,Market,Trading Currency,Open,High,Low,Last,Last Date/Time,Time Zone,Volume,Turnover
ISIN,,,,,,,,,,,,
LU2059756325,AMUNDI AM MSCI EMU,CMUD,Euronext Paris,EUR,40.949,40.949,39.534,39.534,15/04/2020 17:35,CET,0,-
LU2098887263,AMUNDI AM SP500 HE,S500H,Euronext Paris,EUR,41.59,41.59,40.71,40.71,15/04/2020 17:35,CET,0,-
LU2037750168,AMUNDI BEINF US10Y,BINFU,Euronext Paris,USD,48.858,48.858,48.597,48.597,15/04/2020 17:35,CET,0,-
LU2037750168,AMUNDI BEINF US10Y,-,Euronext Amsterdam,EUR,44.551,44.551,44.541,44.541,15/04/2020 17:35,CET,0,-
LU1437018838,AMUNDI EPRA DR,EPRA,Euronext Paris,EUR,50.30,50.30,48.565,48.775,15/04/2020 17:35,CET,11230,553596.094


In [41]:
etf_total.to_excel("data/ETF_Boursorama_top150.xls")